# Peripherals and the Pmod I/O overlay

The Zybo can be extended with additional peripherals by connecting to the Pmod interfaces available on the board. Pmod stands for Peripheral Module, and can be used to connect peripherals and other accessories to the board. 

Pmod peripherals are accessory boards that plug directly into the Pmod port and are available from   [Digilent Inc](http://www.digilentinc.com) and other suppliers. 

## Pmod peripherals

![](./images/Pmods_opt.png)

### Supported peripherals
Pynq current supports a number of Pmods which can be found in the pynq documentation  [ pynq.pmods package](pynq.pmods.html)

As the PMOD interface connects to FPGA pins, any accessory or peripheral supported by the FPGA IO standard (i.e. that works at the correct voltages) that can be wired or connected to a Pmod port can be used.
Pynq current supports a number of Grove peripherals which can be connected to a Pmod port through and interface board. 

Other peripherals can be supported by writing a custom driver using existing drivers as starting points.  

### Pmod Connector
Each Pmod connector has 12 pins (2 rows of 6 pins, where each row has 3.3V (VCC), ground (GND) and 4 data pins giving 8 data pins in total). 

*Close-up of a 12-pin Pmod*

![](images/pmod_closeup_opt.jpeg)

### Pmod pin numbers and assignment

![](images/pmod_pins_opt.png)

Note the Pmod data pins are labelled 0-7. 

### Pmod-Grove Adapter

With a Pmod-to-Grove adapter a wide range of Grove peripherals can be connected to the board.  Each Grove connector has 4 pins, one each for VCC and GND leaving 2 pins on each connector for signal wires.

![](./images/Pmod_Grove_Adapter_opt.jpeg)

### Pmod Pin configuration

Pmods have different pin configurations depending on the number of data pins required. Pmod peripherals with only a single row of pins can be plugged into the top row or the bottom row of the Pmod connector.

Some Pmods use both rows (2x4, 2x6 pins), and in general should be aligned to the upper-left of the connector (to align with VCC and GND).

![](images/tmp2_8pin_opt.jpeg)

## IO Processors
For overlays to be useful, they must provide sufficient functionality, while also providing flexibility to suit a wide range of applications. Flexibility in the overlay is provided through IO Processors (IOPs). An IO Processor is implemented in the Programmable Logic (PL or FPGA fabric) and consists of a MicroBlaze processor, Memory, Timer, IIC, SPI, GPIO, Debug logic and a configurable switch.

![](images/iop.jpg)

The IOP gives flexibility to connect to and control a range of different external accessories without requiring a re-design of the FPGA hardware. The IOP can be used to control a Pmod port, or other interface, or to off-load applications from the main ARM processors in the Zynq PS.

Each IOP has a dedicated memory block for the MicroBlaze instruction and data memory. This memory block is dual-ported, with one port connected to the MicroBlaze, and the other connected to the Cortex-A9 processor. This allows the ARM processor to access the MicroBlaze memory and dynamically write a new program to the MicroBlaze instruction area. The data area can be used for communication and data transfer between the ARM processor and the IOP(s).

Currently, one IOP is dedicated to each Pmod port. Inside the IOP are dedicated interface blocks; Timer, IIC, SPI and GPIO (General Purpose Input/Output). IIC and SPI are specialized interfaces used by many of the available Pmod, Grove and other peripherals, and GPIO can be used to connect to custom interfaces or used as simple inputs and outputs. When a Pmod or other peripheral is plugged in to a port, the configurable switch allows the Pmod signals to be routed internally to the required interface block.

## Zybo Pmod I/O overlay

The Zybo has 6 Pmod connectors. Ports 1-4 as indicated below are connected the Zynq PL. XADC and JF are not included/connected in this overlay.


![](images/zybopmods_opt.jpeg)

The Pmod I/O overlay for the Zybo contains 4 IOPs each connected to one of the PL Pmod ports. This allows control of the Pmod from Python, via the IOP.


![](images/pmodio_overlay_1_opt.png)

A Pmod module is provided in Python, the module must be imported. For Pmod modules, the port that the Pmod is plugged into must also be specified.

## Example: Using the OLED and the ambient light sensor (ALS)

Plug the PmodOLED into Port 1, and PmodALS (Ambient Light Sensor) into the top row of Port 2. (The PmodALS can currently only be used in the top row of a port.)

OLED displaying light reading from ambient light sensor:

![](images/als_oled_Demo_opt.jpeg)

Run the following code to import the OLED module and instantiate it:

In [1]:
from pynq.pmods import PMOD_OLED
oled = PMOD_OLED(4)

Try writing a message to the OLED.

In [2]:
oled.write("Hello World")

In [3]:
oled.clear()

Import the ALS library, create an instance, and read the value from the sensor

In [4]:
from pynq.pmods import PMOD_ALS
als = PMOD_ALS(3)
als.read()

42

Write the value from the ALS to the OLED. The ALS sensor returns an 8-bit value. 0 = Darkest, 255 = Brightest

In [5]:
oled.write("Light value : " + str(als.read()))

In [6]:
from pynq.pmods import PMOD_ALS
import time
als = PMOD_ALS(3)
als.read()

als.set_log_interval_ms(100)

als.start_log() # start logging values at intervals of delay
time.sleep(.2)
als.stop_log()
als.get_log()

[42, 42, 42]

## Example: Using the ADC, DAC and OLED
This example connects the DAC to the ADC. You will write a value to the DAC which the ADC will measure, and also write the value to the OLED for display.

* Set the jumper (JP1) on the PmodAD2 to Ref

Insert the following Pmods:
* DAC to Port 1 (Top Row)
* ADC to Port 2 (Align to the left of the port)
* OLED to Port 4

* Connect a wire from PmodA4 Pin A to PmodAD2 V1

![](./images/adc_dac_demo_opt.jpeg)

* Power on the board, and open the Pynq portal [pynq:9090](http://pynq:9090)
* Run Python script below.

Note the overlay will be reloaded to reset the IOPs if the code at the top of this notebook was run. You can change the value written to the DAC (must be between 0 and 2.0) and rerun the cell. The return value from the ADC and displayed on the LED is the measured value. There may be some small error between this and the value written to the DAC. 

In [7]:
from pynq.pmods import PMOD_OLED, PMOD_ADC, PMOD_DAC
from pynq import Overlay
ol = Overlay("pmod.bit")
ol.download()

dac = PMOD_DAC(1)
adc = PMOD_ADC(2)
oled = PMOD_OLED(4)

dac.write(0.75) # Set the DAC output voltage
value = adc.read() # Measure output of DAC with ADC
oled.write(value) # Display measured value on OLED

For documentation on other supported peripherals, see 

* [Pmods pynq.pmods package](pynq.pmods.html)



## Going further
Scripts can be created and executed from the notebook. 

In this example, the code below will be copied to a script and saved on the board. 

Go back to the Pynq Portal [pynq:9090](http://pynq:9090)

- Navigate to the Getting_Started folder
- Select New > Text File (which opens in a new window)
- Copy and paste the code from the cell below into the file
- Rename the file dac_loop.py, and save it (verify it is in the Getting_Started folder)

In [8]:
from pynq import Overlay
import time
#### dac_loop.py
def dac_loop(DAC_Port = 1, ADC_Port = 2, OLED_Port = 4,Switch_nmbr = 3, delay_secs = 1):
    '''
    DAC -> ADC -> OLED -- LOOP OVER 10 Values
    '''
    ol = Overlay("pmod.bit")
    ol.download()
    # Import PMOD classes from library PythonXilinx (pynq.pmods)
    from pynq.pmods import PMOD_OLED, PMOD_ADC, PMOD_DAC

    from pynq.board import  LED, Switch

    # Match PMOD instances to PMOD Connectors

    # Instantiate OLED
    oled = PMOD_OLED(OLED_Port)

    # Instantiate Digital to Analog Converter
    dac = PMOD_DAC(DAC_Port)

    # Instantiate Analog to Digital Converter
    adc = PMOD_ADC(ADC_Port)

    LED(Switch_nmbr).write(0)
    
    if(Switch(Switch_nmbr).read() == 0):
        print("Switch 3 is off. To run the loop, set Switch 3 to on before running")
    else:
        print("Turn Switch 3 off to stop the program")
    
    while Switch(Switch_nmbr).read():
        LED(Switch_nmbr).write(1)
        for i in range (1, 11):
            dac.write(i/10)
            measured = adc.read()
            time.sleep(.1)
            print('Measurement {} equals: {}'.format(i, measured))
            oled.write(measured)
    LED(Switch_nmbr).write(0)
    

By default, DAC is connected to Port 1 (Top Row), ADC is connected to Port 2, and OLED is connected to Port 4. The function reads  _Switch_ 4 on the board. If the switch is on, it will run a loop that writes to the DAC, reads the value on the ADC and writes to the OLED. This is the same as the previous example, but this time executing from a script, and inside a loop controlled by a switch.

To run this code from the notebook, the cell above can be executed which loads dac_loop() into the notebook

Try executing the cell above, set Switch 3 on the board to "on", and then run the cell below:

In [9]:
dac_loop()

Switch 3 is off. To run the loop, set Switch 3 to on before running


The notebook also supports scripts.  
To run from the script, first Restart the kernel (Kernel > Restart) to clear the dac_loop function previously loaded
Execute the cells below to run the script which will load the dac_loop() function into the notebook, and then uncomment and execute the dac_loop() function in the cell below

In [10]:
#%run dac_loop.py
#dac_loop()